In [2]:
import json
import os
import pandas as pd
import time

from dotenv import load_dotenv
from openrouteservice import client, directions, isochrones, places

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
token_1 = '5b3ce3597851110001cf6248699b88c2149c41bcb3188f454422896f' # raph
token_2 = '5b3ce3597851110001cf62486ee6a1f873a140fe9be5649ca406044d' # raph
token_3 = '5b3ce3597851110001cf6248dc19b6c0177649fe978adbc90d7d99b8' # rapj
token_4 = '5b3ce3597851110001cf6248a037d56d5b27416d9a07d63e77e3d190' # raph

token_6 = '5b3ce3597851110001cf624876aeb78adea8420baafd351196cbd985' # mira
token_7 = '5b3ce3597851110001cf6248367bd3f936244212a4797e74c7b273ea' # ana
token_8 = '5b3ce3597851110001cf624838df250aaf294f39970c4cf365e50280' # daniel
token_9 = '5b3ce3597851110001cf6248ac0814a15df749a7ad88ebe2f35e9c54' # shae
token_10 = '5b3ce3597851110001cf62486727983b54614849a0263ead65e2eeae' # bec
token_11 = '5b3ce3597851110001cf6248300bb89e20f441a496d19ebd08a9ce17' # ruby
token_12 = '5b3ce3597851110001cf62486e654f420ae04b3c9ddc3af4942caa49' # ruby's bf
token_13 = '5b3ce3597851110001cf6248c00dc4500e774a1b8919fba3788779a6' # kelman's friends 1
token_14 = '5b3ce3597851110001cf62485a80770d4c07494dac05ab3619b9c208' # kelman's friends 2
token_15 = '5b3ce3597851110001cf6248d57e2a1220eb4ff5953538e19034b040' # kelman's friends 3
token_16 = '5b3ce3597851110001cf624834fb5b7e9af147f4877c90a0d8784485' # kelman's friends 4
token_17 = '5b3ce3597851110001cf624867cce3b107eb47ff859cec8004c435cf' # kelman's friends 5
token_18 = '5b3ce3597851110001cf62486ee6a1f873a140fe9be5649ca406044d' # raph 1
token_19 = '5b3ce3597851110001cf6248dc19b6c0177649fe978adbc90d7d99b8' # raph 2
token_20 = '5b3ce3597851110001cf624859b80c3d594147d4bb888bc323a446eb' # winona 1
token_21 = '5b3ce3597851110001cf6248a7d915bb377b44f69f942df6e942be0c' # winona 2
token_22 = '5b3ce3597851110001cf624899ceb8cfb4b045cfb0aa7b7cdd27e482' # noah
token_23 = '5b3ce3597851110001cf6248444ceabe552f4bb99053d3f9943750a4' # kelman's friends 6

In [14]:
output_dir = '../data/curated/'

## set variables
load_dotenv()
tokens = [token_1, token_2, token_3, token_4, token_6, 
            token_7, token_8, token_9, token_10, token_11, 
            token_12, token_13, token_14, token_15, token_16, 
            token_17, token_18, token_19, token_20, token_21, 
            token_22, token_23 ]
for i in range(1,24): ## number of keys
    token = os.environ.get(f'token_{i}')
    tokens.append(token)
    
melbourne_cbd = [144.9631, -37.8136] # taken from google maps 
cbd_dict = {}

In [15]:
## read apartment data
property_data = json.load(open( f"{output_dir}cbd_distance.json" ))
cbd_dict = property_data['CBD_distance']

completed = {k:v for k,v in cbd_dict.items() if v is not None}
len(completed)

7686

In [6]:
# load previously run cbd_dict
cbd_dict = json.load(open(f"{output_dir}cbd_distance.json"))

In [88]:
token = tokens[15] #do with more tokens 

## set x and y
x = len(completed)
y = len(completed)+500 ## from smallest API quota
if y > len(property_data['Coordinates'].keys()):
    y = len(property_data['Coordinates'].keys())

for index in range(x,y):
    backwards = list(map(float,property_data['Coordinates'][str(index)][1:-1].split(',')))
    coords = [[backwards[1], backwards[0]]]
    ors = client.Client(key=token)
    
    all = [[str(coords[0][0]),str(coords[0][1])],[str(melbourne_cbd[0]),str(melbourne_cbd[1])]]

    ## get car route
    request = {'coordinates': all,
            'profile': 'driving-car',
            'geometry': 'true',
            'format_out':'geojson'}
    route = ors.directions(**request)
    info = route['features'][0]['properties']['summary']

    ## add information
    cbd_dict[str(index)] = info
    time.sleep(0.5)

In [89]:
# check how many cbd distances have been filled in the property data 
completed = {k:v for k,v in cbd_dict.items() if v is not None}
print(len(completed))

13226


Driveways are longer than 350m to the main road and cannot be detected by ors. Hence, impute as 0

In [44]:
# imputing 0 due to 'Could not find routable point within a radius of 350.0 meters of specified coordinate' error
# the driveway is 450m not on map, api limited to 350m to search for a route
cbd_dict[str(10561)] = 0 

In [64]:
# imputing 0 due to 'Could not find routable point within a radius of 350.0 meters of specified coordinate' error
# the driveway is 500m not on map, api limited to 350m to search for a route
cbd_dict[str(11129)] = 0 

Not on map

In [75]:
# imputing 0 due to 'Could not find routable point within a radius of 350.0 meters of specified coordinate' error
# Benalla residence not forming route
cbd_dict[str(11728)] = 0 
cbd_dict[str(11729)] = 0

Road closed in Rockbank 

In [38]:
# imputing 0 due to 'Could not find routable point within a radius of 350.0 meters of specified coordinate' error
# road closed
cbd_dict[str(10497)] = 0
cbd_dict[str(10581)] = 0
cbd_dict[str(10590)] = 0
cbd_dict[str(10591)] = 0


House on the ocean?

In [86]:
# imputing 0 due to 'Could not find routable point within a radius of 350.0 meters of specified coordinate' error
# house is on the water it seems
cbd_dict[str(12797)] = 0

Save data

In [90]:
property_data['CBD_distance'] = cbd_dict

## write json
json.dump(property_data, open(f"{output_dir}cbd_distance.json", 'w'))

# write csv
data = pd.read_json(f"{output_dir}cbd_distance.json")
data.to_csv(f"{output_dir}cbd_distance.csv")